In [ ]:
import sire as sr
import BioSimSpace as BSS

# Merged Molecules
This section will demonstrate the creation and visualisation of perturbations using BioSimSpace and sire, the system in this case will be a simple ethane &rarr; methanol transformation

In [ ]:
ethane = BSS.Parameters.gaff("CC").getMolecule()
methanol = BSS.Parameters.gaff("CO").getMolecule()
mapping = BSS.Align.matchAtoms(ethane, methanol)
ethane = BSS.Align.rmsdAlign(ethane, methanol, mapping)
merged = BSS.Align.merge(ethane, methanol, mapping)

solvated = BSS.Solvent.tip3p(molecule=merged, box=3 * [3 * BSS.Units.Length.nanometer])

In [ ]:
# Extract the sire system
sire_system = sr.system.System(solvated._sire_object)

In [ ]:
for mol in sire_system.molecules():
    if mol.is_perturbable():
        temp = mol

temp.perturbation().view()

# Custom lambda scheduling
This section will demonstrate the creation and implementation of custom lambda scheduling in sire.

First, create an empty lambda schedule

In [ ]:
l = sr.cas.LambdaSchedule()
l

Add a simple morph to the lambda schedule

In [ ]:
l.add_stage("morphing", (1 - l.lam()) * l.initial() + l.lam() * l.final())

This lambda schedule can then be converted to a dataframe and visualised

In [ ]:
df = l.get_lever_values(to_pandas=True, initial=0, final=1, num_lambda=10)
df.plot()

More complex schedules can be created by adding levers to specific properties using add_lever 

In [ ]:
# Create a new lambda schedule
l_complex = sr.cas.LambdaSchedule()

# Add multiple stages. The charging stages currently act only to set parameters equal to their initial/final values.
l_complex.add_stage("de-charging", l_complex.initial())
l_complex.add_stage(
    "morphing",
    (1 - l_complex.lam()) * l_complex.initial() + l_complex.lam() * l_complex.final(),
)
l_complex.add_stage("re-charging", l_complex.final())

# By adding levers the de-charging and re-charging stages can be applied to specific properties, in this case charge
l_complex.add_lever("charge")
l_complex.set_equation(
    "de-charging", "charge", (1.0 - 0.8 * l_complex.lam()) * l_complex.initial()
)
l_complex.set_equation(
    "re-charging", "charge", (0.2 + 0.8 * l_complex.lam()) * l_complex.final()
)

# We also need to morph the charges scaled by 0.2 (since we scale down to 0.2)
l_complex.set_equation(
    "morphing",
    "charge",
    0.2
    * (
        (1 - l_complex.lam()) * l_complex.initial()
        + l_complex.lam() * l_complex.final()
    ),
)

In [ ]:
df_complex = l_complex.get_lever_values(
    to_pandas=True, initial=0, final=1, num_lambda=10
)
df_complex.plot()